# JupyterLabコンテナからMySQLコンテナへの接続作業メモ
ちょっと苦戦したのでメモしておく。  
一部ChatGPTの出力をそのまま貼り付け。

# ■ ネットワーク設定について
稼働中のコンテナ同士を同じネットワークに属するように設定変更するよりも、  
docker-compose等で設定を追記して立ち上げ直した方が楽。  
下記はmynetworkを作成し、両者のコンテナを同じネットワークに配置して起動させる例。

```yaml:docker-compose.yml
version: '3.7'
services:
  mysql:
    image: mysql:5.7
    environment:
      MYSQL_ROOT_PASSWORD: example
    networks:
      - mynetwork

  jupyterlab:
    image: jupyter/base-notebook
    networks:
      - mynetwork

networks:
  mynetwork:
    driver: bridge
```

# ■ コンテナ起動について
docker-compose upでコンテナを起動したが、  
設定を変えて起動し直したくなったとき、起動し直したいコンテナをdocker rm <コンテナID>で削除しておかないと、  
docker-compose upの際にERROR: 'ContainerConfig'と出てきてしまい、2回目の起動に失敗することがある。  
設定が間違っているように見えるため気づきづらい。

# ■ Jupyter 
docker-compose upでコンテナを起動したが、  
設定を変えて起動し直したくなったとき、起動し直したいコンテナをdocker rm <コンテナID>で削除しておかないと、  
docker-compose upの際にERROR: 'ContainerConfig'と出てきてしまい、2回目の起動に失敗することがある。  
設定が間違っているように見えるため気づきづらい。

## - JupyterLabコンテナでインストールしておくべきもの
Pythonの中でSQLを使うならmysql-connector-pythonだけでも良いが、  
セル内で自由にSQLを実行(マジックコマンドの%%sqlを使う)しようとすると、sqlalchemy等が必要になる。

In [ ]:
# pymysqlあるからいらなかったかも
!pip install mysql-connector-python

[補足] mysqlclientをインストールするために下記が必要  
> apt-get install python3-dev default-libmysqlclient-dev build-essential pkg-config  

参考：https://github.com/PyMySQL/mysqlclient/blob/main/README.md

In [ ]:
# セルから直接SQLを実行できるようにipython-sqlをインストール
!pip install ipython-sql
!pip install sqlalchemy
!pip install pymysql
!pip install mysqlclient

## - MySQLコンテナとの疎通が出来ないときにチェックすべきこと
1. JupyterLabコンテナからMySQLへ接続する際のIPアドレス、MySQLユーザ名、パスワード、対象DBがあっているか
2. リモート接続用ユーザ（JupyterLab→MySQL接続用）のアクセス権限がMySQLサーバ側で設定されているか。